In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
# 시간대별 자전거 수요예측

## 데이터 호출

In [ ]:
df = pd.read_csv('/kaggle/input/london-bike-sharing-dataset/london_merged.csv',parse_dates=['timestamp'])
df.head()

In [ ]:
df['timestamp']

In [ ]:
df.shape

In [ ]:
train = df.iloc[:17000,1:2]
test = df.iloc[17000:17414,1:2] 
# 학습 테스트 데이터를 iloc 메서드로 분류

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
df['cnt'][:17000].plot(figsize=(15,4),legend=True)
df['cnt'][17000:].plot(figsize=(15,4),legend=True)
plt.legend(['train','test'])
plt.title('bike share demand')
plt.show

# 나눈데이터를 plt 로 표시

### 차수측정방법

In [ ]:
from statsmodels.tsa.stattools import pacf #차수추정 방법 
pacf = pacf(df['cnt'], nlags=20 ,method='ols')
print(pacf)

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(pacf, lags=9, method='ols' ,title='pa').show

## 데이터 스케일 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
train_scaled = sc.fit_transform(train)

In [ ]:
train_scaled

In [ ]:
X_train=[]
y_train=[]

for i in range(1,17000):
    X_train.append(train_scaled[i-1:i,0]) # 한시간전으로 shift된 데이터
    y_train.append(train_scaled[i,0]) #원래 데이터

X_train, y_train = np.array(X_train),np.array(y_train)

In [ ]:
X_train.shape

In [ ]:
X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))
X_train.shape

## 모델 구축

### 1. RNN 

 RNN(순환신경망)은 은닉층의 노드에서 활성화 함수를 통해 나온 결과값을 출력층 방향으로도 보내면서, 다시 은닉층 노드의 다음 계산의 입력으로 보내는 특징

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense,Activation 
from keras.layers.recurrent import SimpleRNN

In [ ]:
rnn = Sequential()
rnn.add(SimpleRNN(activation='relu',units=6,input_shape=(1,1)))
rnn.add(Dense(activation='linear',units=1))

In [ ]:
print(rnn.summary())

In [ ]:
rnn.compile(loss='mse',optimizer='adam',metrics=['mse'])

In [ ]:
rnn.fit(X_train,y_train,batch_size=1,epochs=2) #batch_size 몇개 단위로 학습을 시킬것인가?

In [ ]:
inputs = sc.transform(test) #테스트 데이터 스케일링
inputs.shape

In [ ]:
X_test =[]
for i in range(1,415):
    X_test.append(inputs[i-1:i,0])

X_test = np.array(X_test)
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
X_test.shape

In [ ]:
rnn = rnn.predict(X_test)
rnn = sc.inverse_transform(rnn) # 스케일링 한것을 원래의 값으로 되돌려서 넣어준다

In [ ]:
test1 = pd.DataFrame(test)
rnn1 = pd.DataFrame(rnn)

In [ ]:
# 실제값 

test1.plot(figsize=(15,4),legend=True)
plt.legend(['cnt'])
plt.title('bike share demand')
plt.show

In [ ]:
# 예측값 


rnn1.plot(figsize=(15,4),legend=True)
plt.legend(['cnt'])
plt.title('bike share demand')
plt.show

In [ ]:
test =np.array(test)

In [ ]:
# 위 그래프 합치기

plt.figure(figsize=(15,5))
plt.plot(test,marker='.',label='cnt',color='k')
plt.plot(rnn,marker='.',label='RNN',color='r')
plt.legend()

peaktime에서 rnn이 과소평가 한것을 확인 할수 있다.

### 2.LSTM 

LSTM(장단기 메모리)는 은닉층의 메모리 셀에 입력 게이트, 망각 게이트, 출력 게이트를 추가하여 불필요한 기억을 지우고, 기억해야할 것들을 정한다.

In [ ]:
from keras.layers.recurrent import LSTM
lstm = Sequential()
lstm.add(LSTM(units=6,activation='relu',input_shape=(1,1)))
lstm.add(Dense(units=1,activation='linear'))

In [ ]:
print(lstm.summary()) #RNN에 비해 Parameter가 증가한것을 확인할수있다. (오버피팅의 위험이 있다)

In [ ]:
lstm.compile(loss='mse',optimizer='adam',metrics=['mse'])
lstm.fit(X_train,y_train, batch_size=1,epochs=2)

In [ ]:
lstm = lstm.predict(X_test)
lstm = sc.inverse_transform(lstm)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(test,marker='.',label='cnt',color='k')
plt.plot(lstm,marker='.',label='LSTM',color='b')
plt.legend()

### 3. GRU 

GRU(게이트순환유닛) : LSTM의 오버피팅을 방지해주고, 성능은 그대로 유지하는 모델

In [ ]:
from keras.layers.recurrent import GRU

gru = Sequential()
gru.add(GRU(units=6, activation='relu',input_shape=(1,1)))
gru.add(Dense(units=1,activation='linear'))

In [ ]:
print(gru.summary()) #lstm에 비해 파라미터가 줄은것을 확인

In [ ]:
gru.compile(loss='mse',optimizer='adam',metrics=['mse'])
gru.fit(X_train,y_train,batch_size=1,epochs=2)

In [ ]:
gru = gru.predict(X_test)
gru = sc.inverse_transform(gru)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(test,marker='.',label='cnt',color='k')
plt.plot(gru,marker='.',label='GRU',color='g')
plt.legend()

## 결과

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(test,marker='.',label='cnt',color='k')
plt.plot(gru,marker='.',label='GRU',color='g')
plt.plot(lstm,marker='.',label='LSTM',color='b')
plt.plot(rnn,marker='.',label='RNN',color='r')
plt.legend()

In [ ]:
from sklearn.metrics import mean_squared_error
def RMSE(y_test,y_predict):
    return np.sqrt(mean_squared_error(y_test,y_predict))

In [ ]:
print("RNN RMSE",RMSE(test,rnn))
print("LSTM RMSE",RMSE(test,lstm))
print("GRU RMSE",RMSE(test,gru))

LSTM >> GRU >> RNN 순으로 모델 성능을 나타냈다.
Epochs을 2로 설정해서 자세한결과는 얻을수 없었지만 대체적인 성능을 확인할수 있었다.